In [34]:
import os
import Bio
import numpy as np
import pandas as pd

from nimare.extract import download_abstracts, fetch_neuroquery, fetch_neurosynth
from nimare.io import convert_neurosynth_to_dataset



In [45]:
%reload_kedro

[02/29/24 14:55:58] INFO     Resolved project path as:                                              ]8;id=893553;file:///Users/tony/anaconda3/envs/pocs/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=622834;file:///Users/tony/anaconda3/envs/pocs/lib/python3.8/site-packages/kedro/ipython/__init__.py#139\139]8;;\
                             /Users/tony/Documents/phd/courses/PoCS2/spiteful-allegory/work.                       
                             To set a different path, run '%reload_kedro <project_root>'                           

[02/29/24 14:55:58] WARNING  /Users/tony/anaconda3/envs/pocs/lib/python3.8/site-packages/kedro/fram ]8;id=231059;file:///Users/tony/anaconda3/envs/pocs/lib/python3.8/warnings.py\warnings.py]8;;\:]8;id=816302;file:///Users/tony/anaconda3/envs/pocs/lib/python3.8/warnings.py#109\109]8;;\
                             ework/session/session.py:267: KedroDeprecationWarning: ConfigLoader                   
                             will be deprecated in Kedro 0.19. Please use the OmegaConfigLoader                    
                             instead. To consult the documentation for OmegaConfigLoader, see here:                
                             https://docs.kedro.org/en/stable/configuration/advanced_configuration.                
                             html#omegaconfigloader                                                                
                               warnings.warn(                                                                      
                                                                                                                   

                    WARNING  /Users/tony/anaconda3/envs/pocs/lib/python3.8/site-packages/lazy_loade ]8;id=124995;file:///Users/tony/anaconda3/envs/pocs/lib/python3.8/warnings.py\warnings.py]8;;\:]8;id=690293;file:///Users/tony/anaconda3/envs/pocs/lib/python3.8/warnings.py#109\109]8;;\
                             r/__init__.py:78: KedroDeprecationWarning: 'TextDataSet' has been                     
                             renamed to 'TextDataset', and the alias will be removed in                            
                             Kedro-Datasets 2.0.0                                                                  
                               attr = getattr(submod, name)                                                        
                                                                                                                   

                    WARNING  /Users/tony/anaconda3/envs/pocs/lib/python3.8/site-packages/lazy_loade ]8;id=298345;file:///Users/tony/anaconda3/envs/pocs/lib/python3.8/warnings.py\warnings.py]8;;\:]8;id=237390;file:///Users/tony/anaconda3/envs/pocs/lib/python3.8/warnings.py#109\109]8;;\
                             r/__init__.py:78: KedroDeprecationWarning: 'CSVDataSet' has been                      
                             renamed to 'CSVDataset', and the alias will be removed in                             
                             Kedro-Datasets 2.0.0                                                                  
                               attr = getattr(submod, name)                                                        
                                                                                                                   

                    INFO     Kedro project work                                                     ]8;id=236585;file:///Users/tony/anaconda3/envs/pocs/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=612797;file:///Users/tony/anaconda3/envs/pocs/lib/python3.8/site-packages/kedro/ipython/__init__.py#108\108]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=859603;file:///Users/tony/anaconda3/envs/pocs/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=755866;file:///Users/tony/anaconda3/envs/pocs/lib/python3.8/site-packages/kedro/ipython/__init__.py#109\109]8;;\
                             'pipelines'                                                                           

Donwloda NeuroSynth and NeuroQuery datasets (meant for imaging meta analyis) so we can then grab the abstracts for pub med-associated studies. Code taken from:

https://nimare.readthedocs.io/en/latest/auto_examples/01_datasets/02_download_neurosynth.html#sphx-glr-auto-examples-01-datasets-02-download-neurosynth-py

In [35]:
def establish_download_directory():

    out_dir = os.path.abspath("../data/01_raw/neuro-text/")
    os.makedirs(out_dir, exist_ok=True)

    return out_dir

out_dir = establish_download_directory()

In [36]:
def download_and_convert_neurodata(out_dir, dataset:str, email="example@example.edu"):
    '''Save full dataset but return abstracts.'''

    if dataset == "neurosynth":
        files = fetch_neurosynth(
            data_dir=out_dir,
            version="7",
            overwrite=False,
            source="abstract",
            vocab="terms",
        )
    elif dataset == "neuroquery": 
        files = fetch_neuroquery(
            data_dir=out_dir,
            version="1",
            overwrite=False,
            source="combined",
            vocab="neuroquery6308",
            type="tfidf",
        )
    ndb = files[0]

    n_dset = convert_neurosynth_to_dataset(
        coordinates_file=ndb["coordinates"],
        metadata_file=ndb["metadata"],
        annotations_files=ndb["features"],
        )
    n_dset = download_abstracts(n_dset, email)

    # save full dataset
    fname = f"{dataset}_dataset_with_abstracts.pkl.gz"
    n_dset.save(os.path.join(out_dir, fname))

    return n_dset.texts

In [37]:
# neurosynth_texts = download_and_convert_neurodata(out_dir, 'neurosynth')
# neuroquery_texts = download_and_convert_neurodata(out_dir, 'neuroquery')

In [46]:
neurosynth_text = catalog.load('neurosynth_text')
neurosynth_text['source'] = 'neurosynth'

neuroquery_text = catalog.load('neuroquery_text')
neuroquery_text['source'] = 'neuroquery'

text = pd.concat([neurosynth_text, neuroquery_text])
text.head()

[02/29/24 14:56:01] INFO     Loading data from 'neurosynth_text' (CSVDataset)...                ]8;id=518101;file:///Users/tony/anaconda3/envs/pocs/lib/python3.8/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=653886;file:///Users/tony/anaconda3/envs/pocs/lib/python3.8/site-packages/kedro/io/data_catalog.py#502\502]8;;\

[02/29/24 14:56:02] INFO     Loading data from 'neuroquery_text' (CSVDataset)...                ]8;id=960548;file:///Users/tony/anaconda3/envs/pocs/lib/python3.8/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=59488;file:///Users/tony/anaconda3/envs/pocs/lib/python3.8/site-packages/kedro/io/data_catalog.py#502\502]8;;\

,id,study_id,contrast_id,abstract,source
0,10022492-1,10022492,1,A fundamental characteristic of working memory...,neurosynth
1,10022494-1,10022494,1,Electrophysiological studies on monkeys have b...,neurosynth
2,10022496-1,10022496,1,Most functional imaging studies of the auditor...,neurosynth
3,10051677-1,10051677,1,Positron emission tomography studies were cond...,neurosynth
4,10191322-1,10191322,1,The cortical organization of language in bilin...,neurosynth


In [47]:
text.shape

(27830, 5)

In [49]:
pd.unique(text['study_id']).shape

(19148,)